Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [132]:
from random import choices, choice, randint, random
from copy import copy
from dataclasses import dataclass
from tqdm import tqdm
import numpy as np

import lab9_lib

In [148]:
fitness = lab9_lib.make_problem(2)
some = [0] * 1000
for i in range(1, 1000, 2):
    some[i] = 1
print(fitness(some))
# for n in range(10):
#     ind = choices([0, 1], k=1000)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
# fitnesses = sorted((some[s :: 2] for s in range(2)), reverse=True)
# print(fitnesses)
# print(fitness.calls)

0.5


In [134]:
POPULATION_SIZE = 25
OFFSPRING_SIZE = 10
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .30
INDIVIDUAL_SIZE = 1000
ACCEPTANCE = 0.001

In [149]:

class Individual:
    def __init__(self, fitness, genotype: np.ndarray):
        self.fitness = fitness
        self.genotype = genotype

    @staticmethod
    def generate(size = INDIVIDUAL_SIZE):
        return Individual(fitness = None, genotype = np.random.choice(a = np.array([True, False]), size = size))

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = Individual(None, ind.genotype.copy())
    pos = randint(0, INDIVIDUAL_SIZE-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    return offspring

def mutation(g):
    offspring = Individual(None, ind.genotype.copy())
    point = randint(0, INDIVIDUAL_SIZE - 1)
    offspring.genotype = offspring.genotype[:point] + (1 - offspring.genotype[point],) + offspring.genotype[point + 1 :]
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, INDIVIDUAL_SIZE-1)
    offspring = Individual(fitness=None,
                           genotype=np.concatenate( (ind1.genotype[:cut_point], ind2.genotype[cut_point:]) ))
    assert offspring.genotype.shape[0] == INDIVIDUAL_SIZE
    return offspring

def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    mask1 = np.random.choice([True, False], size = INDIVIDUAL_SIZE)
    mask2 = np.logical_not(mask1)
    assert np.logical_or(mask1, mask2).sum() == INDIVIDUAL_SIZE
    tmp1 = np.logical_and(ind1.genotype, mask1)
    tmp2 = np.logical_and(ind2.genotype, mask2)
    new_genotype = np.logical_or(tmp1, tmp2)
    return Individual(None, new_genotype)

def or_xover(ind1: Individual, ind2: Individual) -> Individual:
    new_genotype = np.logical_or(ind1.genotype, ind2.genotype)
    return Individual(None, new_genotype)

In [136]:
def generate_population(fitness_func, size = POPULATION_SIZE):
    population = [ Individual.generate() for _ in range(size)]
    for ind in population:
        ind.fitness = fitness_func(ind.genotype)
        print(ind.fitness)
    return population

# 1 - Problem

In [137]:
fitness = lab9_lib.make_problem(1)
population = generate_population(fitness, size = POPULATION_SIZE)
cache = {tuple(ind.genotype): ind.fitness for ind in population} # memo object to avoid recomputing the fitness
best_fitness = 0.0

for generation in range(10_000):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            # o = one_cut_xover(p1, p2)
            o = uniform_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        tmp = tuple(i.genotype)
        if tmp in cache:
            i.fitness = cache[tmp]
        else:
            i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda x: x.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    # every 10 generations test improvements
    if generation % 20 == 0:
        curr_best_fitness = population[0].fitness
        if abs(best_fitness - curr_best_fitness) < ACCEPTANCE:
            break
        best_fitness = max(best_fitness, curr_best_fitness)
    if generation % 50 == 0:
        print(f'Gen: {generation} --- fitness: {population[0].fitness}')
        print(f'{len(population)}')
        # print("-------------- Fitness ---------------")
        # for el in population:
        #     print(f'{el.fitness} ---- {sum(el.genotype)}')
        # print("\n\n")

0.524
0.502
0.508
0.51
0.487
0.51
0.503
0.495
0.511
0.512
0.497
0.519
0.491
0.522
0.507
0.501
0.501
0.493
0.487
0.483
0.481
0.495
0.5
0.503
0.477
Gen: 0 --- fitness: 0.526
25
Gen: 50 --- fitness: 0.594
25
Gen: 100 --- fitness: 0.625
25
Gen: 150 --- fitness: 0.653
25
Gen: 200 --- fitness: 0.677
25
Gen: 250 --- fitness: 0.705
25
Gen: 300 --- fitness: 0.726
25
Gen: 350 --- fitness: 0.751
25
Gen: 400 --- fitness: 0.774
25
Gen: 450 --- fitness: 0.797
25
Gen: 500 --- fitness: 0.817
25
Gen: 550 --- fitness: 0.835
25
Gen: 600 --- fitness: 0.851
25
Gen: 650 --- fitness: 0.866
25
Gen: 700 --- fitness: 0.881
25
Gen: 750 --- fitness: 0.894
25
Gen: 800 --- fitness: 0.903
25
Gen: 850 --- fitness: 0.919
25
Gen: 900 --- fitness: 0.93
25
Gen: 950 --- fitness: 0.942
25
Gen: 1000 --- fitness: 0.951
25
Gen: 1050 --- fitness: 0.957
25


In [138]:
print(f"fitness Calls: {fitness.calls}")
best = population[0]
print(fitness(list(best.genotype)))
print(f'{fitness(list(best.genotype)):.2%}')

fitness Calls: 10634
0.957
95.70%


# 2 - Problem

In [161]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .70
INDIVIDUAL_SIZE = 1000
ACCEPTANCE = 0.001

In [162]:
fitness = lab9_lib.make_problem(2)
population = generate_population(fitness, size = POPULATION_SIZE)
cache = {tuple(ind.genotype): ind.fitness for ind in population} # memo object to avoid recomputing the fitness
best_fitness = 0.0

for generation in range(100_000):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            # o = one_cut_xover(p1, p2)
            o1 = mutate(p1)
            o2 = mutate(p2)
            o = uniform_xover(o1, o2)
            offspring.append(o)

    for i in offspring:
        tmp = tuple(i.genotype)
        if tmp in cache:
            i.fitness = cache[tmp]
        else:
            i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda x: x.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    # every 10 generations test improvements
    if generation % 20 == 0:
        curr_best_fitness = population[0].fitness
        if abs(best_fitness - curr_best_fitness) < ACCEPTANCE:
            break
        best_fitness = max(best_fitness, curr_best_fitness)
    if generation % 50 == 0:
        print(f'Gen: {generation} --- fitness: {population[0].fitness}')
        print(f'{len(population)}')
        # print("-------------- Fitness ---------------")
        # for el in population:
        #     print(f'{el.fitness} ---- {sum(el.genotype)}')
        # print("\n\n")

0.2206
0.2144
0.22640000000000002
0.2368
0.2459
0.2284
0.2268
0.2332
0.2185
0.2301
0.2305
0.2378
0.2308
0.21430000000000002
0.22290000000000001
0.2246
0.22490000000000002
0.2238
0.21969999999999998
0.24130000000000001
0.2384
0.22469999999999998
0.22790000000000002
0.2324
0.2333
0.224
0.2352
0.2226
0.2324
0.226
Gen: 0 --- fitness: 0.518
30
Gen: 50 --- fitness: 0.554
30
Gen: 100 --- fitness: 0.574
30
Gen: 150 --- fitness: 0.6
30
Gen: 200 --- fitness: 0.63
30
Gen: 250 --- fitness: 0.658
30
Gen: 300 --- fitness: 0.68
30
Gen: 350 --- fitness: 0.704
30
Gen: 400 --- fitness: 0.724
30
Gen: 450 --- fitness: 0.748
30
Gen: 500 --- fitness: 0.766
30
Gen: 550 --- fitness: 0.784
30
Gen: 600 --- fitness: 0.798
30
Gen: 650 --- fitness: 0.814
30
Gen: 700 --- fitness: 0.826
30
Gen: 750 --- fitness: 0.842
30
Gen: 800 --- fitness: 0.854
30
Gen: 850 --- fitness: 0.862
30
Gen: 900 --- fitness: 0.876
30
Gen: 950 --- fitness: 0.886
30
Gen: 1000 --- fitness: 0.892
30
Gen: 1050 --- fitness: 0.906
30
Gen: 1100 -

In [163]:
print(f"fitness Calls: {fitness.calls}")
best = population[0]
print(fitness(list(best.genotype)))
print(f'{fitness(list(best.genotype)):.2%}')

fitness Calls: 22450
0.912
91.20%
